# Molecular Simulation 
Carrying out molecular dynamics (MD) and Monte-Carlo (MC) simulations using a (trained) potential. 

In [1]:
from utils import set_env
# set_env('.env')

In [2]:
import os
os.environ["JAX_ENABLE_X64"] = "1"
os.environ["JAX_PLATFORM_NAME"] = "cpu" 
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false" 

In [3]:
from jaxip.datasets import RunnerDataset
from jaxip.potentials import NeuralNetworkPotential
from jaxip.simulation import (
    MDSimulator, 
    BrendsenThermostat, 
    MCSimulator,
    run_simulation
)
from jaxip.atoms import Structure
from jaxip.units import units as units

import matplotlib.pylab as plt
from pathlib import Path
from ase import Atoms
from ase.visualize import view
import ase.io
import jax.numpy as jnp

In [4]:
base_dir = Path('LJ')

## Data

In [7]:
d = 6  # Angstrom
uc = Atoms('He', positions=[(d/2, d/2, d/2)], cell=(d, d, d))
s0 = Structure.from_ase(uc.repeat((7, 7, 7)))

# d = 10  # Angstrom
# uc = Atoms('Ar', positions=[(d/2, d/2, d/2)], cell=(d, d, d))
# s0 = Structure.create_from_ase(uc.repeat((7, 7, 7)))

atoms = s0.to_ase()
# view(atoms, viewer='x3d') # ase, ngl

## Potential

In [8]:
from jaxip.types import Array
import jax
from functools import partial
from jaxip.simulation.lj import LJPotential

# He
ljpot = LJPotential(
    sigma=2.5238 * units.FROM_ANGSTROM,  # Bohr
    epsilon=4.7093e-04 * units.FROM_ELECTRON_VOLT,  # Hartree
    r_cutoff=6.3095 * units.FROM_ANGSTROM,  # 2.5 * sigma
)

# Ar
# ljpot = LJPotential(
#     sigma=3.405 * units.FROM_ANGSTROM,                       # Bohr
#     epsilon=0.01032439284 * units.FROM_ELECTRON_VOLT,        # Hartree
#     r_cutoff=8.5125 * units.FROM_ANGSTROM,                   # 2.5 * sigma
# )

## Molecular Dynamics (MD)

In [9]:
# v0 = MDSimulator.generate_random_velocity(temperature=300.0, mass=s0.mass, seed=2023)
# brendsen = BrendsenThermostat(target_temperature=300.0, time_constant=50.0 * units.FROM_FEMTO_SECOND)

md = MDSimulator(
    potential=ljpot,
    initial_structure=s0,
    time_step=0.5 * units.FROM_FEMTO_SECOND,
    temperature=300, # K
    # initial_velocity=v0,
    # thermostat=brendsen
)

In [10]:
md.get_pressure()

Array(6.51659916e-07, dtype=float64)

In [11]:
md.get_total_energy()

Array(0.48832374, dtype=float64)

In [ ]:
# Warmp up
# run_simulation(md)

# %timeit run_simulation(md, num_steps=1, output_freq=-1)

In [12]:
run_simulation(md, num_steps=1000, output_freq=100, filename="md.xyz")

0          time[ps]:0.00000    Temp[K]:299.95012  Etot[Ha]:0.4883237353    Epot[Ha]:-0.0003923654   Pres[kb]:0.19173   
100        time[ps]:0.05000    Temp[K]:180.48925  Etot[Ha]:0.2937277968    Epot[Ha]:-0.0003477770   Pres[kb]:0.11535   
200        time[ps]:0.10000    Temp[K]:340.79198  Etot[Ha]:0.5549463578    Epot[Ha]:-0.0003143736   Pres[kb]:0.21783   
300        time[ps]:0.15000    Temp[K]:332.48107  Etot[Ha]:0.5414355956    Epot[Ha]:-0.0002839803   Pres[kb]:0.21252   
400        time[ps]:0.20000    Temp[K]:339.34123  Etot[Ha]:0.5526201298    Epot[Ha]:-0.0002768727   Pres[kb]:0.21688   
500        time[ps]:0.25000    Temp[K]:325.52335  Etot[Ha]:0.5301128543    Epot[Ha]:-0.0002703262   Pres[kb]:0.20803   
600        time[ps]:0.30000    Temp[K]:323.53786  Etot[Ha]:0.5268878850    Epot[Ha]:-0.0002602887   Pres[kb]:0.20678   
700        time[ps]:0.35000    Temp[K]:298.66455  Etot[Ha]:0.4863645128    Epot[Ha]:-0.0002569646   Pres[kb]:0.19088   
800        time[ps]:0.40000    Temp[K]:3

In [13]:
md.get_pressure() * units.TO_KILO_BAR

Array(0.20212924, dtype=float64)

In [14]:
md.positions * units.TO_ANGSTROM

Array([[ 1.11870489,  4.24844493,  2.75083361],
       [ 0.56430649,  2.14820432,  8.35313146],
       [ 3.85854755,  3.8140108 , 13.61781432],
       ...,
       [36.87880675, 37.60983005, 23.85054836],
       [37.4298655 , 40.55351055, 30.59379365],
       [37.90171642, 35.95951402, 38.96970769]], dtype=float64)

In [15]:
atoms = md.get_structure().to_ase()
# view(atoms, viewer='x3d') # ase, ngl

## Monte Carlo (MC)

In [16]:
mc = MCSimulator(
    potential=ljpot,
    initial_structure=s0,
    temperature=300, # K
    translate_step=0.3 * units.FROM_ANGSTROM,
    movements_per_step=10,
)

In [17]:
# Warmp up
# run_simulation(mc)

# %timeit run_simulation(mc, num_steps=1, output_freq=-1)

In [18]:
run_simulation(mc, num_steps=1000, output_freq=100, filename=Path("mc.xyz"))

0          Temp[K]:300.0000000000  Epot[Ha]:-0.0003923654   


/home/hossein/miniconda3/envs/jaxip/lib/python3.10/site-packages/jax/_src/ops/scatter.py:89: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float64 to dtype=float32. In future JAX releases this will result in an error.
  warnings.warn("scatter inputs have incompatible types: cannot safely cast "


100        Temp[K]:300.0000000000  Epot[Ha]:-0.0003693695   
200        Temp[K]:300.0000000000  Epot[Ha]:-0.0003808577   
300        Temp[K]:300.0000000000  Epot[Ha]:-0.0004226280   
400        Temp[K]:300.0000000000  Epot[Ha]:-0.0004656510   
500        Temp[K]:300.0000000000  Epot[Ha]:-0.0005302971   
600        Temp[K]:300.0000000000  Epot[Ha]:-0.0005892853   
700        Temp[K]:300.0000000000  Epot[Ha]:-0.0006245603   
800        Temp[K]:300.0000000000  Epot[Ha]:-0.0006825552   
900        Temp[K]:300.0000000000  Epot[Ha]:-0.0007941456   
1000       Temp[K]:300.0000000000  Epot[Ha]:-0.0008474013   


In [19]:
mc.positions * units.TO_ANGSTROM

Array([[ 2.3410995,  2.7073226,  3.6566348],
       [ 3.868568 ,  2.184813 , 10.019266 ],
       [ 4.2380447,  4.090038 , 14.468224 ],
       ...,
       [38.596302 , 39.81191  , 26.777454 ],
       [38.508682 , 37.05469  , 34.859104 ],
       [40.550755 , 38.24894  , 40.7568   ]], dtype=float32)

In [20]:
atoms = mc.get_structure().to_ase()
# view(atoms, viewer='x3d') # ase, ngl